## ns-3 RIT Scenario: Single Run and Statistics

This notebook provides a streamlined workflow for running an ns-3 RIT MAC scenario and
automatically aggregating the resulting logs.

1. Run ns-3 simulations directly from Python
2. Parse generated logs and compute key metrics
3. Output results as a DataFrame and export them to CSV

It is intended for quick validation and evaluation of a single RIT simulation scenario.


## Simulation configuration (↓↓ edit here ↓↓)

In [ ]:
%load_ext autoreload
%autoreload 2
from common.config_utils import SimulationConfig

# === Simulation parameters (edit here) ===
simulation_params = {
    "BI": 5000,          # Beacon interval [ms]
    "TWD": 5000,         # TX wait duration [ms]
    "DWD": 10,           # Data wait duration [ms]
    "Nodes": -1,         # Use default node count
    "Days": 1,           # Simulation duration
    "DR": 0,             # Data rate / traffic option
    "AppPacketSize": 8,  # Application payload size [bytes]
    "Seed": 1,           # Random seed
    "DataCsma": "false",
    "DataPreCs": "false",
    "BeaconCsma": "false",
    "BeaconPreCs": "false",
    "ContinuousTx": "false",
    "BeaconRandomize": "false",
    "CompactRitDataRequest": "true",
    "BeaconAck": "false",
    "Placement": "center",
    "Density": "middle",
    "App": "periodic",
}
base_script = "scratch/rit-grid-converge.cc"
config = SimulationConfig(simulation_params, base_script)

print(f"NS-3 working directory: {config.ns3_working_dir}")
print(f"Module name: {config.module_name}")
print(f"Parameter directory: {config.parameter_dir}")
print(f"Generated ns-3 Command: {config.ns3_command}")

In [ ]:
# reload util modules
import importlib
import common.config_utils
importlib.reload(common.config_utils)

In [ ]:
from common.ns3_utils import run_ns3_simulation

# --- run single ns-3 simulation ---
run_ns3_simulation(config)

## Results

In [ ]:
import os
import pandas as pd

from common.io_utils import read_log, find_node_dirs
from common.aggregate_utils import (
    aggregate_app_summary,
    aggregate_mac_summary,
    aggregate_phy_summary,
    aggregate_scenario_summary,
)
from common.log_constants import MAC_LOG_FILES, APP_TXLOG, APP_RXLOG

# --- User configuration ---
base_dir = config.ns3_working_dir + "/logs"
parameter_dir = config.parameter_dir
module_name = config.module_name
scenario_type = config.scenario_type

# --- Detect number of nodes ---
# If Nodes = -1, automatically infer node count from log directories
if config.simulation_params["Nodes"] == -1:
    log_path = os.path.join(base_dir, parameter_dir)
    if os.path.exists(log_path):
        node_dirs = [d for d in os.listdir(log_path) if d.startswith("node-")]
        node_ids = [int(d.split("-")[1]) for d in node_dirs]
        actual_nodes = max(node_ids) + 1 if node_ids else 0
        print(f"Auto-detected {actual_nodes} nodes from log directory")
    else:
        actual_nodes = 0
        print(f"Log directory not found: {log_path}")
else:
    actual_nodes = config.simulation_params["Nodes"]

# --- Node roles ---
MAC_NODES = list(range(actual_nodes))
APP_SEND_NODES = list(range(1, actual_nodes))  # all except sink
APP_RECV_NODE = 0                              # sink node

print(f"Using {actual_nodes} nodes")
print(f"MAC_NODES = {MAC_NODES}")
print(f"APP_SEND_NODES = {APP_SEND_NODES}")

# --- Application-layer aggregation ---
app_df = aggregate_app_summary(
    APP_SEND_NODES,
    APP_RECV_NODE,
    APP_TXLOG,
    APP_RXLOG,
    base_dir,
    parameter_dir,
)
print("=== Application-layer summary ===")
display(app_df)

# --- MAC-layer aggregation ---
mac_df = aggregate_mac_summary(
    MAC_NODES,
    MAC_LOG_FILES,
    base_dir,
    parameter_dir,
)
print("=== MAC-layer summary ===")
display(mac_df)

# --- PHY-layer aggregation ---
phy_df = aggregate_phy_summary(
    MAC_NODES,
    base_dir,
    parameter_dir,
)
print("=== PHY-layer summary ===")
display(phy_df)

# --- Scenario-level summary ---
scenario_summary = aggregate_scenario_summary(app_df, phy_df)
scenario_summary_df = pd.DataFrame([scenario_summary])
display(scenario_summary_df)

# --- Save summaries ---
summary_dir = os.path.join(base_dir, parameter_dir, "summary")
os.makedirs(summary_dir, exist_ok=True)

app_df.to_csv(config.get_summary_path("app-summary.csv"), index=False)
mac_df.to_csv(config.get_summary_path("mac-summary.csv"), index=False)
phy_df.to_csv(config.get_summary_path("phy-summary.csv"), index=False)